###HOUSE PRICE PREDICTION WITh MLFLOW 

In [5]:
import pandas as pd 
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.datasets import fetch_california_housing
from mlflow.models import infer_signature
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error 
housing=fetch_california_housing()

In [6]:
print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [9]:
housing['feature_names']

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [13]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['price']=housing['target']
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# train and test split , model training and hyperparamter MLflow experiments 


In [14]:
from urllib.parse import urlparse##
X=data.drop('price',axis=1)
y=data['price']

In [19]:
def hyperparameter_tuning(X_train,y_train,params):
    rf=RandomForestRegressor()
    
    grid_search=GridSearchCV(rf,param_grid=params,cv=3,n_jobs=-1,verbose=2,scoring='neg_mean_squared_error')
    grid_search.fit(X_train,y_train)
    return grid_search

In [ ]:
from math import e
from flask import url_for
from matplotlib.pyplot import grid

#train and test split , model training and hyperparamter MLflow experiments
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
from mlflow.models import infer_signature
signature=infer_signature(X_train,y_train)
##define parameter grid 

params={
    'n_estimators':[100,200],
    'max_depth':[5,10,20],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4]
}

#start the mlflow experiments 
with mlflow.start_run():
    #perform hyperparameter tuning
    grid_search=hyperparameter_tuning(X_train,y_train,params)
    #get the best model
    best_model=grid_search.best_estimator_
    #evalute the model
    y_pred=best_model.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)

    ##log best parameters and metrics
    mlflow.log_params("best_n_estimator",grid_search.best_params_['n_estimators'])
    mlflow.log_params("max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_params("min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_params("min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse )


    #tracking url
    
# set MLflow tracking server (local or remote)
    '''Basically, urlparse is used so your code can adapt automatically to whether you are using local MLflow tracking or a remote MLflow server.'''
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")    
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store!="file":
        mlflow.sklearn.log_model(best_model,'model',registered_model_name="RandomForestRegressorHousePrice",signature=signature)
    else:
        mlflow.sklearn.log_model(best_model,'model',signature=signature)#if it is a file store menas no need to registery in the server based on the tracking url type store
    print("best params:",grid_search.best_params_)
    print("best model",best_model)
    print("mse:",mse)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


AttributeError: 'str' object has no attribute 'items'